<a href="https://colab.research.google.com/github/wakhidrf/Information-Security-Portfolio/blob/main/Spam_or_Ham%3F_Build_an_Email_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 0. Go to the Ham or Spam dataset [website](http://www2.aueb.gr/users/ion/data/enron-spam/index.html) and download the dataset.

In [1]:
!wget https://cdn.theforage.com/vinternships/companyassets/Sj7temL583QAYpHXD/JiwEkbBq8pFwMRYLc/1635444722719/enron1.zip
!unzip -qq enron1.zip

--2024-01-27 00:33:47--  https://cdn.theforage.com/vinternships/companyassets/Sj7temL583QAYpHXD/JiwEkbBq8pFwMRYLc/1635444722719/enron1.zip
Resolving cdn.theforage.com (cdn.theforage.com)... 18.65.39.85, 18.65.39.31, 18.65.39.102, ...
Connecting to cdn.theforage.com (cdn.theforage.com)|18.65.39.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5783105 (5.5M) [application/zip]
Saving to: ‘enron1.zip’

enron1.zip          100%[===================>]   5.51M  7.35MB/s    in 0.8s    

2024-01-27 00:33:48 (7.35 MB/s) - ‘enron1.zip’ saved [5783105/5783105]



Step 1. Traverse the dataset and create a Pandas dataframe. This is already done for you. You should recognize Pandas from task 1.

In [2]:
import pandas as pd
import os

def read_spam():
    category = 'spam'
    directory = './enron1/spam'
    return read_category(category, directory)

def read_ham():
    category = 'ham'
    directory = './enron1/ham'
    return read_category(category, directory)

def read_category(category, directory):
    emails = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename), 'r') as fp:
            try:
                content = fp.read()
                emails.append({'name': filename, 'content': content, 'category': category})
            except:
                print(f'skipped {filename}')
    return emails

ham = read_ham()
spam = read_spam()

df = pd.DataFrame.from_records(ham)
df = df.append(pd.DataFrame.from_records(spam))

skipped 2140.2004-09-13.GP.spam.txt
skipped 3304.2004-12-26.GP.spam.txt
skipped 2042.2004-08-30.GP.spam.txt
skipped 4142.2005-03-31.GP.spam.txt
skipped 2526.2004-10-17.GP.spam.txt
skipped 5105.2005-08-31.GP.spam.txt
skipped 3364.2005-01-01.GP.spam.txt
skipped 4350.2005-04-23.GP.spam.txt
skipped 4201.2005-04-05.GP.spam.txt
skipped 2248.2004-09-23.GP.spam.txt
skipped 2649.2004-10-27.GP.spam.txt
skipped 4566.2005-05-24.GP.spam.txt
skipped 2698.2004-10-31.GP.spam.txt
skipped 1414.2004-06-24.GP.spam.txt
skipped 0754.2004-04-01.GP.spam.txt


<ipython-input-2-49292689b560>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame.from_records(spam))


Step 2. Data cleaning is a critical part of machine learning. You and I can recognize that 'Hello' and 'hello' are the same word but a machine does not know this a priori. Therefore, we can 'help' the machine by conducting such normalization steps for it. Write a function `preprocessor` that takes in a string and replaces all non alphabet characters with a space and then lowercases the result.

In [3]:
import re

def preprocessor(e):
    return re.sub('[^A-Za-z]', ' ', e).lower()

Step 3. We will now train the machine learning model. All the functions that you will need are imported for you. The instructions explain how the work and hint at which functions to use. You will likely need to refer to the scikit learn documentation to see how exactly to invoke the functions. It will be handy to keep that tab open.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# The CountVectorizer converts a text sample into a vector (think of it as an array of floats).
# Each entry in the vector corresponds to a single word and the value is the number of times the word appeared.
# Instantiate a CountVectorizer. Make sure to include the preprocessor you previously wrote in the constructor.
vectorizer = CountVectorizer(preprocessor=preprocessor)

# Use train_test_split to split the dataset into a train dataset and a test dataset.
# The machine learning model learns from the train dataset.
# Then the trained model is tested on the test dataset to see if it actually learned anything.
# If it just memorized for example, then it would have a low accuracy on the test dataset and a high accuracy on the train dataset.
X_train,X_test,y_train,y_test = train_test_split(df["content"],df["category"],test_size=0.2,random_state=1)

# Use the vectorizer to transform the existing dataset into a form in which the model can learn from.
# Remember that simple machine learning models operate on numbers, which the CountVectorizer conveniently helped us do.
X_train_df = vectorizer.fit_transform(X_train)

# Use the LogisticRegression model to fit to the train dataset.
# You may remember y = mx + b and Linear Regression from high school. Here, we fitted a scatter plot to a line.
# Logistic Regression is another form of regression.
# However, Logistic Regression helps us determine if a point should be in category A or B, which is a perfect fit.
model = LogisticRegression()
model.fit(X_train_df,y_train)

# Validate that the model has learned something.
# Recall the model operates on vectors. First transform the test set using the vectorizer.
# Then generate the predictions.
X_test_df = vectorizer.transform(X_test)
y_pred = model.predict(X_test_df)

# We now want to see how we have done. We will be using three functions.
# `accuracy_score` tells us how well we have done.
# 90% means that every 9 of 10 entries from the test dataset were predicted accurately.
# The `confusion_matrix` is a 2x2 matrix that gives us more insight.
# The top left shows us how many ham emails were predicted to be ham (that's good!).
# The bottom right shows us how many spam emails were predicted to be spam (that's good!).
# The other two quadrants tell us the misclassifications.
# Finally, the `classification_report` gives us detailed statistics which you may have seen in a statistics class.
print(f'Accuracy:\n{accuracy_score(y_test,y_pred)}\n')
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y_pred)}\n')
print(f'Detailed Statistics:\n{classification_report(y_test,y_pred)}\n')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy:
0.9844961240310077

Confusion Matrix:
[[729  13]
 [  3 287]]

Detailed Statistics:
              precision    recall  f1-score   support

         ham       1.00      0.98      0.99       742
        spam       0.96      0.99      0.97       290

    accuracy                           0.98      1032
   macro avg       0.98      0.99      0.98      1032
weighted avg       0.98      0.98      0.98      1032




Step 4.

In [5]:
# Let's see which features (aka columns) the vectorizer created.
# They should be all the words that were contained in the training dataset.
features = vectorizer.get_feature_names_out()

# You may be wondering what a machine learning model is tangibly. It is just a collection of numbers.
# You can access these numbers known as "coefficients" from the coef_ property of the model
# We will be looking at coef_[0] which represents the importance of each feature.
# What does importance mean in this context?
# Some words are more important than others for the model.
# It's nothing personal, just that spam emails tend to contain some words more frequently.
# This indicates to the model that having that word would make a new email more likely to be spam.
importance = model.coef_[0]

# Iterate over importance and find the top 10 positive features with the largest magnitude.
# Similarly, find the top 10 negative features with the largest magnitude.
# Positive features correspond to spam. Negative features correspond to ham.
# You will see that `http` is the strongest feature that corresponds to spam emails.
# It makes sense. Spam emails often want you to click on a link.
l = list(enumerate(importance))
print()
l.sort(key=lambda e: e[1], reverse=True)
for i,imp in l[:10]:
    print(imp, features[i])
print()
l.sort(key=lambda e: -e[1], reverse=True)
for i,imp in l[:10]:
    print(imp, features[i])


1.0044362456278242 http
0.8176885681071172 prices
0.7534654533601661 no
0.7461598743726818 removed
0.7267990653549244 paliourg
0.7265242987677138 money
0.713340948588752 pain
0.7021934812555215 hello
0.6604174958432891 only
0.6464351702087976 here

-1.5794746802145332 attached
-1.5759056713601534 enron
-1.485565636119017 daren
-1.3598683323538934 thanks
-1.2818984200245023 doc
-1.227009256033395 pictures
-1.1982884032602719 xls
-1.172552906695079 neon
-1.129481269928728 meter
-1.1232421117631208 hpl


All Done!